### 第8回レポート課題その1

In [1]:
!pip install gensim

In [2]:
FILE_ID = "0B7XkCwpI5KDYNlNUTTlSS21pQmM"
FILE_NAME = "GoogleNews-vectors-negative300.bin.gz"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=$FILE_ID' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$FILE_ID" -O $FILE_NAME && rm -rf /tmp/cookies.txt

--2020-07-26 12:26:40--  https://docs.google.com/uc?export=download&confirm=v62S&id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
Resolving docs.google.com (docs.google.com)... 173.194.79.100, 173.194.79.138, 173.194.79.113, ...
Connecting to docs.google.com (docs.google.com)|173.194.79.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-38-docs.googleusercontent.com/docs/securesc/8b1tv88p51e066en95560v87n44mb4eh/ek1hh5raaogjr185oer2c87s08qmvkgl/1595766375000/06848720943842814915/05963322287483552980Z/0B7XkCwpI5KDYNlNUTTlSS21pQmM?e=download [following]
--2020-07-26 12:26:42--  https://doc-0k-38-docs.googleusercontent.com/docs/securesc/8b1tv88p51e066en95560v87n44mb4eh/ek1hh5raaogjr185oer2c87s08qmvkgl/1595766375000/06848720943842814915/05963322287483552980Z/0B7XkCwpI5KDYNlNUTTlSS21pQmM?e=download
Resolving doc-0k-38-docs.googleusercontent.com (doc-0k-38-docs.googleusercontent.com)... 74.125.143.132, 2a00:1450:4013:c03::84
Connecting to doc-0k-38-d

In [3]:
!wget http://130.153.158.5/~inaba/data70.zip

--2020-07-26 12:27:04--  http://130.153.158.5/~inaba/data70.zip
Connecting to 130.153.158.5:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 392010 (383K) [application/zip]
Saving to: ‘data70.zip’

data70.zip          100%[===================>] 382.82K  1.30KB/s    in 7.0s    

2020-07-26 12:27:12 (55.0 KB/s) - ‘data70.zip’ saved [392010/392010]



In [4]:
!unzip data70.zip

Archive:  data70.zip
  inflating: test.txt                
  inflating: train.txt               
  inflating: valid.txt               


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
import gensim
import re
import torch
import numpy as np

w2v_path = './GoogleNews-vectors-negative300.bin.gz' #todo GoogleNews-vectors-negative300.binのパスを指定
data_path = './' #todo ファイル出力ディレクトリを指定
data_path2 = '/content/drive/My Drive/data/'

w2v_model = gensim.models.KeyedVectors.load_word2vec_format(w2v_path, binary=True) #todo gensimでword2vecのモデルを読み込む

category2num = {"b": 0, "t": 1, "e": 2, "m": 3}


# タイトルを受け取り，単語ベクトルの平均を返す関数
def get_feature(title):
    word_list =  title.split(' ')  #todo タイトルをスペースで分割
    vec_list = []
    for word in word_list:
        try:
            vec = w2v_model[word] #todo wordを意味するベクトルを取得
        except KeyError:
            vec = [0] #todo すべての要素が0のベクトルを代入
        vec_list.append(vec)    
    vec_np = np.array(vec_list[1]) # numpyのarrayに変換
    feature = np.sum(vec_np, axis=0) / len(vec_np)  # 平均ベクトルを計算
    return feature


def get_data(fname):
    label_list = []
    feature_list = []
    with open(fname) as f:
        for line in f:
            if not line:
                continue
            data = line.split('\t')
            title = data[1] #todo dataからタイトルを取り出す
            feature = get_feature(title) 
            feature_list.append(feature)
            label = category2num[data[0]]
            label_list.append(label)
    features = torch.tensor(feature_list) #todo feature_listをtensorに変換
    labels = torch.tensor(label_list) #todo label_listをtensorに変換
    return features, labels


train_x, train_y = get_data(data_path + "train.txt")
valid_x, valid_y = get_data(data_path + "valid.txt")
test_x, test_y = get_data(data_path + "test.txt")

# 保存
torch.save(train_x, data_path2 + "train_x.pt")
torch.save(train_y, data_path2 + "train_y.pt")
torch.save(valid_x, data_path2 + "valid_x.pt")
torch.save(valid_y, data_path2 + "valid_y.pt")
torch.save(test_x, data_path2 + "test_x.pt")
torch.save(test_y, data_path2 + "test_y.pt")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


#### 感想
今回用いた関数はこれからも必要になる機会はあると思うのでよく復習しておきたい.